In [ ]:
import numpy as np

from bikewheelcalc import BicycleWheel, Rim, Hub, Spoke
import bikewheellib as bl

import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

In [ ]:
R = 0.300
GJ = 25.0
EI1 = 100.
EI2 = GJ / 0.2

w = BicycleWheel()
w.hub = Hub(diam1=0.05, width1=0.025)
w.rim = Rim(radius=R, area=100e-6,
            I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
            young_mod=69.0e9, shear_mod=26.0e9)

In [ ]:
# Radial stiffness vs. Pippard Number
rr = bl.ModeMatrix(w, N=4*len(wh.spokes))

ns_0 = 36.
ds_0 = 1.8e-3

n_spk = np.arange(6, 37, 2)

K_rad_sm = np.zeros(n_spk.shape)
K_rad_ds = np.zeros(n_spk.shape)
l_SP = np.zeros(n_spk.shape)  # Smith-Pippard number

for i, ns in enumerate(n_spk):
    
    # Calculate spoke diameter to keep ns*As constant
    ds = ds_0 * np.sqrt(ns_0 / ns)
    w.lace_radial(n_spokes=int(ns), diameter=ds, young_mod=200e9, offset=0.00)
    w.apply_tension(0.01)
    
    rr = bl.ModeMatrix(w, N=4*len(wh.spokes))
    
    B_v0 = rr.B_theta(0., comps=[1])
                       
    K_sm = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)
    K_ds = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=False)
    
    F_ext = rr.calc_F_ext([0.], np.array([[0., 1., 0., 0.]]))

    K_rad_sm[i] = 1./(B_v0.dot(np.linalg.solve(K_sm, F_ext)))
    K_rad_ds[i] = 1./(B_v0.dot(np.linalg.solve(K_ds, F_ext)))
    
    # Calculate Smith-Pippard number
    l_SP[i] = bl.calc_Pn_rad(w)

In [ ]:
w.lace_cross(n_cross=3, n_spokes=24, diameter=1.8e-3, young_mod=200e9, offset=0.00)

# Lateral deflection under radial load
w.apply_tension(0.1)
rr = bl.ModeMatrix(w, N=4*len(wh.spokes))

K_sm = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)
K_ds = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=False)

B_0 = rr.B_theta(0., comps=[0, 1])

# Radial deflection under radial point load
F_ext = rr.calc_F_ext([0.], np.array([[0., 1., 0., 0.]]))

d_rad_sm = np.linalg.solve(K_sm, F_ext)
d_rad_ds = np.linalg.solve(K_ds, F_ext)

th = np.linspace(-np.pi, np.pi, 4*len(wh.spokes))

B_u = rr.B_theta(th, comps=[0])
B_v = rr.B_theta(th, comps=[1])

# Lateral deflection under radial point load, as radial load is moved around the wheel
u_sm = np.zeros(th.shape)
u_ds = np.zeros(th.shape)
v_ds = np.zeros(th.shape)
for i, t in enumerate(th):
    F_ext = rr.calc_F_ext([t], np.array([[0., 1., 0., 0.]]))
    
    d_sm = np.linalg.solve(K_sm, F_ext)
    d_ds = np.linalg.solve(K_ds, F_ext)
    B_t = rr.B_theta(t, comps=[0, 1])
    
    u_sm[i] = B_t.dot(d_sm)[0]
    u_ds[i] = B_t.dot(d_ds)[0]
    v_ds[i] = B_t.dot(d_ds)[1]

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(6.5, 2.))


# Radial stiffness vs. Smith-Pippard number
ax[0].plot([0., 2.], [K_rad_sm[0]/1e6, K_rad_sm[0]/1e6], 'C0--')
ax[0].plot(l_SP, K_rad_ds / 1e6, 'C1')


ax[0].set_xlim([0., 1.5])
ax[0].set_xticks(np.arange(0., 1.6, 0.25))
ax[0].set_xticklabels(['0', '', '0.5', '', '1', '', '1.5'])
ax[0].set_xlabel('Spokes per char. len')

ax[0].set_ylim([0., 8.])
ax[0].set_ylabel(r'\$K_{rad}\$ [kN/mm]')

# ax[0].set_xticks([])
# ax[0].set_yticks([])


# Radial deflection for smeared and discrete spokes
ax[1].plot(B_v.dot(d_rad_sm) / B_0.dot(d_rad_ds)[1], 'C0--')
ax[1].plot(B_v.dot(d_rad_ds) / B_0.dot(d_rad_sm)[1], 'C1')

ax[1].set_xticks([]);
ax[1].set_xlabel(r'\$\theta\$')

ax[1].set_ylim([-0.25, 1.1])
ax[1].set_yticks([-0.25, 0., 0.25, 0.5, 0.75, 1.])
ax[1].set_yticklabels(['', '0', '', '', '', '1'])
ax[1].set_ylabel('Radial deflection')


# Lateral deflection under radial load as the wheel is rotated
ax[2].plot(u_sm / v_ds, 'C0--')
ax[2].plot(u_ds / v_ds, 'C1')

ax[2].set_xticks([])
ax[2].set_xlabel('Radial load position')

ax[2].set_ylim([-0.5, 0.5])
ax[2].set_yticks(np.arange(-0.5, 0.51, 0.25))
ax[2].set_yticklabels(['-0.5','', '0', '', '+0.5'])
ax[2].set_ylabel('Lateral deflection')

plt.tight_layout()
plt.savefig('../figs/stress_analysis/_python_continuum_vs_discrete.pdf')